In [3]:
import os
import numpy as np
import pandas as pd


ModuleNotFoundError: No module named 'numpy'

In [3]:
# WRITE PATH TO DATA DIRECTORY BELOW 

data_path = '../data/redclassic/'

files = []
data = {}
files_received = []
load_files = []
driver_files = []

try:
    for i in os.listdir(data_path):
        if os.path.isfile(os.path.join(data_path,i)):
            files.append(i)
except FileNotFoundError:
    print("PLEASE MAKE SURE THE DATA PATH LEADS TO THE DATA DIRECTORY WHERE THE DATA FILES ARE LOCATED\n")   
except NameError:
    print("PLEASE INSERT A VALID PATH STRING TO THE DATA DIRECTORY\n")

if '.DS_Store' in files:
    files.remove('.DS_Store')
for file in files:
    data[file] = pd.read_csv(data_path + file, low_memory=False)

for file in files:
    try:
        data[file]
        files_received.append(file)
        if 'driver' in file.lower():
            driver_files.append(file)
        else:
            load_files.append(file)
    except KeyError:
        print(f'** The {file} file is missing from the data folder **'.upper())        

if len(driver_files) == 1:
    print(f"Received {len(driver_files)} driver file")
else:
    print(f"Received {len(driver_files)} driver files")
    
if len(load_files) == 1:
    print(f"Received {len(load_files)} load file")
else:
    print(f"Received {len(load_files)} load files")

if len(files_received) == 0:
    print("\n!!! NO FILES RECEIVED !!!")
    
for file in files_received:
    print(f"{file} - {type(data[file])}")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
# print(data['RED_CLASSIC OPT DYNAMICS 2021 Q1.csv'].info())
# print(data['OPT DYNAMICS 2021 Q2.csv'].info())
# print(data['OPT DYNAMICS 2021 Q3.csv'].info())
# print(data['OPT DYNAMICS 2021 Q4.csv'].info())

# CYCLE THROUGH LOADS FILES
for file in loads_files:
    load_df = data[file]
    

In [ ]:
# CYCLE THROUGH DRIVERS FILES

for file in drivers_files:
    driver_df = data[file]
    